In [ ]:
!pip install scikit-learn nltk

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk

nltk.download('punkt')  # Descargar el tokenizer de oraciones si es necesario

# Función para cargar todos los textos de una carpeta
def cargar_textos_de_carpeta(carpeta):
    textos = {}
    for root, dirs, files in os.walk(carpeta):
        for filename in files:
            if filename.endswith('.txt'):  # Solo procesar archivos .txt
                ruta_completa = os.path.join(root, filename)
                with open(ruta_completa, 'r', encoding='utf-8') as file:
                    textos[ruta_completa] = file.read()
    return textos

# Función para calcular la similitud entre dos textos
def calcular_similitud(text1, text2):
    vectorizer = TfidfVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0][1]

# Función para alinear todas las transcripciones con los textos base y guardar en CSV, incluyendo la ruta de audio
def alinear_y_guardar_resultados_con_audio(carpeta_audio_base, carpeta_textos_base, carpeta_transcripciones, carpeta_salida_csv):
    # Cargar textos base y transcripciones
    textos_base = cargar_textos_de_carpeta(carpeta_textos_base)

    for usuario in os.listdir(carpeta_transcripciones):
        ruta_usuario = os.path.join(carpeta_transcripciones, usuario)
        if os.path.isdir(ruta_usuario):
            transcripciones = cargar_textos_de_carpeta(ruta_usuario)

            resultados = []

            for ruta_transcripcion, transcripcion_texto in transcripciones.items():
                mejor_similitud = 0
                mejor_texto_base = None
                ruta_mejor_texto_base = None

                for ruta_texto_base, texto_base_texto in textos_base.items():
                    similitud = calcular_similitud(transcripcion_texto, texto_base_texto)

                    if similitud > mejor_similitud:
                        mejor_similitud = similitud
                        mejor_texto_base = os.path.basename(ruta_texto_base)  # Nombre del archivo base
                        ruta_mejor_texto_base = ruta_texto_base  # Ruta completa del texto base

                # Definir la ruta de audio correspondiente
                ruta_audio = os.path.join(carpeta_audio_base, usuario, os.path.basename(ruta_transcripcion).replace('.txt', '.wav'))

                resultados.append({
                    'nombre_carpeta': usuario,
                    'texto_transcripcion': os.path.basename(ruta_transcripcion),
                    'ruta_texto_transcripcion': ruta_transcripcion,
                    'texto_base': mejor_texto_base,
                    'ruta_texto_base': ruta_mejor_texto_base,
                    'porcentaje_similitud': round(mejor_similitud * 100, 2),  # Convertir a porcentaje
                    'ruta_audio': ruta_audio
                })

            # Guardar resultados en un CSV
            df_resultados = pd.DataFrame(resultados)
            nombre_csv = f'resultados_{usuario}.csv'
            ruta_csv = os.path.join(carpeta_salida_csv, nombre_csv)
            os.makedirs(carpeta_salida_csv, exist_ok=True)  # Asegurarse de que la carpeta de salida existe
            df_resultados.to_csv(ruta_csv, index=False)
            print(f"Resultados guardados para {usuario} en {ruta_csv}")

# Rutas a las carpetas (ajusta estas rutas según tu sistema)
carpeta_textos_base = '/content/drive/MyDrive/Proyecto_ASR_AG/Texto_Base'
carpeta_transcripciones = '/content/drive/MyDrive/Proyecto_ASR_AG/Texto_ASR'
carpeta_salida_csv = '/content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv'
carpeta_audio_base = '/content/drive/MyDrive/Proyecto_ASR_AG/Proyecto_AG'

# Ejecutar la función
alinear_y_guardar_resultados_con_audio(carpeta_audio_base, carpeta_textos_base, carpeta_transcripciones, carpeta_salida_csv)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Resultados guardados para ATMA en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_ATMA.csv
Resultados guardados para .ipynb_checkpoints en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_.ipynb_checkpoints.csv
Resultados guardados para SSF en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_SSF.csv
Resultados guardados para RRYR en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_RRYR.csv
Resultados guardados para PVM en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_PVM.csv
Resultados guardados para LPJM en /content/drive/MyDrive/Proyecto_ASR_AG/resultados_csv/resultados_LPJM.csv
